In [ ]:
!pip install langchain openai


ERROR: Could not find a version that satisfies the requirement sqlite3 (from versions: none)
ERROR: No matching distribution found for sqlite3


In [ ]:
import os
from langchain.chat_models import ChatOpenAI

# Set your OpenAI API key
os.environ["OPENAI_API_KEY"] = ""

In [ ]:
import os
import sqlite3
import streamlit as st
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.agents import initialize_agent, Tool, AgentType
from langchain.schema import SystemMessage, HumanMessage
from langchain.prompts import PromptTemplate
from langgraph.graph import StateGraph, END
from dataclasses import dataclass

# -----------------------
# 🔹 SETUP
# -----------------------

# Connect to SQLite Movie Database
conn = sqlite3.connect('movie_trailers.db')
cursor = conn.cursor()

# Initialize LangChain LLM
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.2)

# Setup Memory for Conversational Context
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
    input_key="user_query",
    output_key="response"
)

# -----------------------
# 🔹 DEFINE FUNCTIONS
# -----------------------

# ✅ **1️⃣ Convert Natural Language to SQL**
def generate_sql(state):
    """Convert user input to SQL while remembering past conversations."""
    query = state.user_query

    # Retrieve conversation history
    past_conversation = memory.load_memory_variables({})
    history = past_conversation.get("chat_history", "")

    prompt = PromptTemplate.from_template(
        "You are an SQL expert. Given this conversation history:\n{history}\n\n"
        "Convert the user's latest question into an SQL query:\n\n{query}\n\nSQL:"
    )

    messages = [
        SystemMessage(content="You are a SQL assistant."),
        HumanMessage(content=prompt.format(history=history, query=query))
    ]
    
    sql_query = llm.invoke(messages).content.strip()  # ✅ Fixed `.invoke(messages)`

    # Store query in memory
    memory.save_context({"user_query": query}, {"response": sql_query})

    return QueryState(user_query=query, sql_query=sql_query)


# ✅ **2️⃣ Execute SQL Query**
def query_movie_db(state):
    """Executes an SQL query against the movie_trailer database."""
    sql_query = state.sql_query
    try:
        cursor.execute(sql_query)
        results = cursor.fetchall()
        return QueryState(user_query=state.user_query, sql_query=sql_query, db_results=results if results else "No results found.")
    except Exception as e:
        return QueryState(user_query=state.user_query, sql_query=sql_query, db_results=f"Error executing query: {str(e)}")


# ✅ **3️⃣ Format Results**
def format_results(state):
    """Format query results into natural language."""
    results = state.db_results
    return QueryState(user_query=state.user_query, sql_query=state.sql_query, db_results=f"Results:\n{results}" if results else "No relevant data found.")


# ✅ **4️⃣ General Movie Knowledge (LLM)**
def answer_general_movie_question(question: str) -> str:
    """Uses GPT to answer general movie-related questions."""
    messages = [
        SystemMessage(content="You are a movie expert. Answer questions about movies, actors, and awards."),
        HumanMessage(content=question)
    ]
    response = llm.invoke(messages).content  # ✅ Fixed `.invoke(messages)`
    return response

# -----------------------
# 🔹 SET UP LangGraph PIPELINE
# -----------------------
@dataclass
class QueryState:
    user_query: str = ""
    sql_query: str = ""
    db_results: str = ""

graph = StateGraph(QueryState)
graph.add_node("generate_sql", generate_sql)
graph.add_node("execute_sql", query_movie_db)
graph.add_node("format_response", format_results)

graph.set_entry_point("generate_sql")
graph.add_edge("generate_sql", "execute_sql")
graph.add_edge("execute_sql", "format_response")
graph.add_edge("format_response", END)

app = graph.compile()

# -----------------------
# 🔹 DEFINE LANGCHAIN AGENT
# -----------------------

# ✅ **Database Query Tool**
db_tool = Tool(
    name="MovieDB",
    func=lambda query: app.invoke(QueryState(user_query=query)).db_results,  # ✅ Fixed incorrect lambda return
    description="Queries the movie_trailer database and returns movie-related information."
)

# ✅ **General Movie Knowledge Tool**
llm_tool = Tool(
    name="MovieKnowledge",
    func=answer_general_movie_question,
    description="Answers general movie-related questions, such as movie plots, actor details, and award history."
)

# ✅ **Initialize LangChain Agent**
tools = [db_tool, llm_tool]

agent = initialize_agent(
    tools=tools,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    llm=llm,
    memory=memory,
    verbose=False  # ✅ Reduced verbosity for better performance
)

# -----------------------
# 🔹 STREAMLIT UI
# -----------------------

# ✅ **Ensure session state for conversation history**
if "chat_history" not in st.session_state:
    st.session_state.chat_history = ""

st.title("🎬 Movie Insights Chatbot")

# User Input
user_query = st.text_area("Ask about movies:", "")

# Submit Button
if st.button("Get Insights"):
    if user_query.strip():
        # Run the AI pipeline
        response = agent.run(user_query)

        # ✅ **Store conversation history properly**
        st.session_state.chat_history += f"User: {user_query}\nAgent: {response}\n"

        # ✅ **Show chat history**
        st.subheader("📜 Chat History")
        st.text(st.session_state.chat_history)

        # ✅ **Show insights**
        st.subheader("📊 Insights")
        st.write(response)
        
    else:
        st.warning("Please enter a query.")


2025-02-14 12:17:20.977 
  command:

    streamlit run c:\Users\onyek\anaconda3\envs\myenv\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
